# PPD: OpenMP - *Worksharing*

Hélio - DC/UFSCar - 2023

# Paralelismo com divisão de trabalho: *worksharing* 

Além de permitir a **replicação** da execução de trechos de código, OpenMP possui construções para **dividir** as execuções desses trechos. Essas construções são chamadas *worksharing* e sempre se aplicam dentro de uma região paralela definida pela diretiva ***parallel***.

Construções do tipo *worksharing* **não** geram novas *threads*, mas se aplicam às *threads* do time associado à região paralela atual.

Não há uma barreira no início de uma construção desse tipo, mas uma é inserida automaticamente ao seu final.

Tipos de construções para divisão de carga (*worksharing*) (para linguagem C):

* **for** : dividem as iterações de um *loop* entre as *threads* do time. Representam o paralelismo de dados.
* **sections** : dividem o trabalho em regiões explicitamente definidas. Cada seção é executada por uma *thread*. Pode ser usada para representar o paralelismo funcional.
* **single** : serializa um trecho de código, que é executado por apenas 1 *thread* do time.

# *Single*: executando um trecho de código por apenas uma *thread*

Aplicada dentro de uma região paralela, a diretiva **single** especifica que o código associado deve ser executado por apenas uma *thread* no time. Fica a critério da API determinar qual será, não sendo necessariamente a *master*.

Normalmente, o uso da diretiva *single* é útil no tratamento de seções de código que não são *thread safe* (como E/S) e devem ser executadas por uma *thread* apenas.

```
#pragma omp parallel
{
  ...
  #pragma omp single [clause ...]  newline
    structured_block
  ...
}
```
```
cláusulas:  private (list)
            firstprivate (list)
            nowait
```
*Theads* no time que não executam a diretiva *single* esperam no final do código associado, exceto se a cláusula *nowait* for especificada.


In [ ]:
%%writefile s1.c

#include <stdio.h>    // para printf()
#include <stdlib.h>   // para random()
#include <omp.h>      // para omp_get_thread_num()

int
main()
{
   int a, i;
   // ...
 
#pragma omp parallel shared(a) private(i)
{
   // ...                     // Todas as threads do time executam essa parte
  #pragma omp single          // Só uma thread do time vai executar esse bloco
  {
    printf("thread %d executou código no bloco single\n\n", omp_get_thread_num());
    srand(time(NULL));
  }                           // Uma barreira é usada aqui, se a cláusula nowait não for especificada
 
  // ...                      // restante do código do bloco paralelo: todas as threads executam
  printf("thread %d executando bloco paralelo\n",omp_get_thread_num());
  a = rand();
  // ...                      // Todas as threads do time executam essa parte
} //                          // Fim do bloco paralelo
  // ...                      // Só a master thread prossegue
 
  return(0);
}

Overwriting s1.c


In [ ]:
!gcc s1.c -o s1 -fopenmp && ./s1

# *Parallel for*: dividindo as iterações de um *loop*

A diretiva *for* especifica que as iterações do *loop* imediatamente abaixo devem ser executadas em paralelo, **dividindo-as** entre as *threads* do time.

Emitida dentro de uma região paralela, esta diretiva deve ser sucedida especificamente por um comando **for**.

```
#pragma omp for [clause ...] newline
   Cláusulas:
     schedule (type [,chunk])
     ordered
     private (list)
     first private (list)
     last private (list)
     shared (list)
     reduction (operator: list)
     collapse (n)
     nowait
```
Cláusulas:

* **schedule**: determina como as iterações do *loop* serão divididas entre as *threads* do time.
  * ***static***: iterações divididas em blocos de tamanho *chunk*.
  * ***dynamic***: iterações divididas em blocos de tamanho *chunk* e alocadas dinamicamente entre as *threads*, à medida que terminam as iterações atribuídas anteriormente.
  * ***guided***: número de iterações atribuído em cada rodada é calculado em função das iterações restantes divididas pelo número de *threads*, sendo o resultado decrescido de *chunk*.
  * ***runtime***: decisão de atribuição é realizada somente em tempo de execução, usando a política que tiver sido definida pela variável de ambiente OMP_SCHEDULE.
  * ***auto***: decisão de atribuição é delegada ao compilador ou software em tempo de execução.
* ***nowait***: se usada, esta cláusula indica que *threads* não devem ser sincronizadas no fim do loop paralelo.
* ***ordered***: indica que as iterações do loop devem ser executadas em sequência como se fossem trataras em um programa serial.
* ***collapse***: indica quantos *loops* em um aninhamento de *loops* (*nested loops*) devem ser agrupados (*collapsed*) em um bloco de iteração maior dividido de acordo com a cláusula *schedule*. 


**For canônico**

Para poder transformar um *loop* sequencial em paralelo é preciso que o compilador OpenMP seja capaz de verificar que o sistema em tempo de execução terá as informações necessárias para determinar o número de iterações ao avaliar a cláusula de controle. 

Resumidamente, a definição das iterações do comando *for* deve ser clara, para que o suporte em tempo de execução saiba **quantas** e **quais** são as iterações, para poder dividi-las entre as *threads* do time corrente!

*For loops* devem possuir a seguinte forma canônica:

```
for ( index = start; index {<,<=,>=,>} end;
     { indx++, ++indx, indx--, --indx, indx+=inc, index -= inc, indx= indx+inc, indx=inc+indx, indx=indx-inc} )
```
O exemplo a seguir ilustra o uso da diretiva ***for***:

In [ ]:
%%writefile s2.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <omp.h>

#define MAX 20

int
main(int argc, char **argv)
{
    int i, id, num_it, vet[MAX];

    srand(time(NULL));
    num_it = rand() % MAX;
         
  #pragma omp parallel num_threads(4) private(id)  // a variável de controle do for precisa ser privada. Isso é feito automaticamente, contudo.
  {
    // Todas as threads do time executam esse trecho do bloco de código de maneira replicada
    id = omp_get_thread_num();
   
    // Construção worksharing (for) divide as iterações entre as threads do time
    // Variável de controle do for (i) é feita privada para cada thread, automaticamente!
    // Diretiva for deve aparecer dentro de uma região paralela
    #pragma omp for         // O único comando permitido na linha abaixo da diretiva for é um for :-)
    for (i=0; i < num_it; i++) {       
        printf("Thread %d tratando iteração %d\n", id, i);
        vet[i] = 2 * i;
    }
    // todas as threads replicam esse trecho de código, fora do for, mas dentro da região paralela

   } // fim da região paralela 

    return 0;
}

Overwriting s2.c


In [ ]:
!gcc s2.c -o s2 -fopenmp && ./s2

Thread 2 tratando iteração 8
Thread 2 tratando iteração 9
Thread 2 tratando iteração 10
Thread 2 tratando iteração 11
Thread 1 tratando iteração 4
Thread 1 tratando iteração 5
Thread 1 tratando iteração 6
Thread 1 tratando iteração 7
Thread 3 tratando iteração 12
Thread 3 tratando iteração 13
Thread 3 tratando iteração 14
Thread 0 tratando iteração 0
Thread 0 tratando iteração 1
Thread 0 tratando iteração 2
Thread 0 tratando iteração 3


**Forma compacta de declaração do *paralell for***

Quando o paralelismo desejado no programa é apenas para divisão das iterações de um comando *for*, é possível usar a declaração compacta da *diretiva for*:

```
  ...
  #pragma omp parallel for      // Linha seguinte DEVE ser um comando for 
  for (i=0; i < NUM; i++) {     // (*)
    printf("Thread %d tratando iteração %d\n", omp_get_thread_num(), i); 
    vet[i] = 2 * i;  
  } 
  ...
```
(\*) É importante observar que, para que cada *thread* do time execute uma parte das iterações, cada uma deve ter sua própria cópia da variável de controle do *loop*. Isso significa que, neste caso, a variável ***i***, usada como índice do comando *for*, deve ser **privada**. O compilador com suporte a OpenMP, contudo, faz com que essa variável seja privada automaticamente.


**Controlando a divisão das iterações**

Uma vez entendido que as iterações de um comando for paralelo serão **divididas** entre as *threads* do time, podemos pensar em como essa divisão ocorrerá.

Por padrão, a divisão é em bloco, de forma que cada *thread* será encarregada de 1 / N das iterações. Vale lembrar que cada *thread* num time tem um número lógico que vai de 0 (para a *thread master*) a N-1.

Assim, a *thread* ***i*** vai executar as iterações  **i * (1/N) .. (i+1) * (1/N) -1**

Por exemplo, com 12 iterações e 4 *threads*, a thread 0 vai executar as iterações 0, 1 e 2, a *thread* 1 vai executar as iterações 3, 4 e 5, a *thread* 2 vai executar 6, 7 e 8, e, por fim, a *thread* 3 vai executar as iterações 9, 10 e 11.

O compilador está atento, contudo, e consegue tratar os **casos em que essa divisão não é exata**! Neste caso, comumente, as "resto da divisão inteira" primeiras *threads* recebem **uma iteração a mais cada uma**.

A forma de divisão das iterações pode ser controlada pelo programador, por exemplo, usando a cláusula *schedule* na primitiva for. 

De acordo com as especificações, o comportamento da divisão de iterações pode variar como segue:

* *STATIC: Loop iterations are divided into pieces of size chunk and then statically assigned to threads. If chunk is not specified, the iterations are evenly (if possible) divided contiguously among the threads.*
* *DYNAMIC: Loop iterations are divided into pieces of size chunk, and dynamically scheduled among the threads; when a thread finishes one chunk, it is dynamically assigned another. The default chunk size is 1.*
* *GUIDED: For a chunk size of 1, the size of each chunk is proportional to the number of unassigned iterations divided by the number of threads, decreasing to 1. For a chunk size with value k (greater than 1), the size of each chunk is determined in the same way with the restriction that the chunks do not contain fewer than k iterations (except for the last chunk to be assigned, which may have fewer than k iterations). The default chunk size is 1.*
* *RUNTIME: The scheduling decision is deferred until runtime by the environment variable OMP_SCHEDULE. It is illegal to specify a chunk size for this clause.*
* *AUTO: The scheduling decision is delegated to the compiler and/or runtime system.*

A definição da política de escalonamento das iterações pode ocorrer de três formas: 
* via **cláusula schedule** na diretiva *for*, 
* via chamada à função ***omp_set_schedule(omp_sched_tkind, intchunk_size)*** e 
* via variável de ambiente **OMP_SCHEDULE**.

Vejamos um programa que realiza a divisão das iterações via primitiva *for* numa região paralela.

In [ ]:
%%writefile pfor.c

#include <stdio.h>
#include <omp.h>

#define NUM 12

int 
main(int argc, char **argv) 
{
  int i, ind, vet[NUM];

 #pragma omp parallel private(ind)
 {
   ind = omp_get_thread_num();

   #pragma omp for schedule(runtime)
   for (i=0; i < NUM; i++) {
     printf("Thread %d executando iteracao %d\n", ind, i);
     vet[i] = 2 * i;
   }
 }

 return 0;
}

Overwriting pfor.c


Vejamos o resultado da execução usando diferentes formas de particionamento das iterações.

In [ ]:
! gcc pfor.c -o pfor -fopenmp 

In [ ]:
! OMP_SCHEDULE=static ./pfor | sort 

Thread 0 executando iteracao 0
Thread 0 executando iteracao 1
Thread 0 executando iteracao 2
Thread 0 executando iteracao 3
Thread 0 executando iteracao 4
Thread 0 executando iteracao 5
Thread 1 executando iteracao 10
Thread 1 executando iteracao 11
Thread 1 executando iteracao 6
Thread 1 executando iteracao 7
Thread 1 executando iteracao 8
Thread 1 executando iteracao 9


In [ ]:
! OMP_SCHEDULE=dynamic ./pfor | sort

Thread 0 executando iteracao 1
Thread 1 executando iteracao 0
Thread 1 executando iteracao 10
Thread 1 executando iteracao 11
Thread 1 executando iteracao 2
Thread 1 executando iteracao 3
Thread 1 executando iteracao 4
Thread 1 executando iteracao 5
Thread 1 executando iteracao 6
Thread 1 executando iteracao 7
Thread 1 executando iteracao 8
Thread 1 executando iteracao 9


In [ ]:
! OMP_NUM_THREADS=4 OMP_SCHEDULE=guided ./pfor | sort

Thread 0 executando iteracao 0
Thread 0 executando iteracao 1
Thread 0 executando iteracao 10
Thread 0 executando iteracao 11
Thread 0 executando iteracao 2
Thread 0 executando iteracao 6
Thread 0 executando iteracao 7
Thread 0 executando iteracao 8
Thread 0 executando iteracao 9
Thread 1 executando iteracao 3
Thread 1 executando iteracao 4
Thread 1 executando iteracao 5


**Resumindo?**

Se o trecho com maior processamento no seu programa é um ***loop for***, cujas iterações podem ser executadas em paralelo, a paralelização deste código com OpenMP pode ser feita em uma linha! 

Além disso, a forma de divisão das iterações pode ser ajustada cada vez que você for executar o programa!

Ou seja, para experimentar com diferentes formas de divisão das iterações e com diferentes números de *threads*, nem é preciso recompilar o programa!

```
  ...
  #pragma omp parallel for
  for( ...; ...; ...) {
    ...
  }
  ...
```

Que tal pensarmos no impacto do algoritmo de atribuição das iterações?

Afinal, por que fazer a atribuição de outra forma que não em blocos (N_it / num_threads) ?

In [ ]:
%%writefile para-for.c

#include <stdio.h>
#include <unistd.h>

#define N 1000

int
main()
{
  int i;
 
  #pragma omp parallel for
  for (i=0; i< N; i++) 
    // simula algum processamento que depende do índice da iteração sendo calculada
    // quanto maior a iteração, mais processamento.
    // Se divisão for estática, thread que ficar com o último bloco de iterações terá mais trabalho
    // usleep( i * 40 );
    for (int j=0; j< 1000 * i * 30; j++);
  return 0;
}

Overwriting para-for.c


In [ ]:
! if $( ! apt list time 2>&1 | grep "installed" ) ; then apt install time &> /dev/null; fi 
! if [ ! para-for -nt para-for.c ]; then gcc -Wall para-for.c -o para-for -fopenmp; fi 

! echo static:
! OMP_SCHEDULE=static time -p ./para-for
! echo; echo dynamic:
! OMP_SCHEDULE=dynamic time -p ./para-for
! echo; echo guided:
! OMP_SCHEDULE=guided time -p ./para-for
# ! echo; echo guided, chunk=2:
# ! export OMP_SCHEDULE="guided,10"
# ! OMP_SCHEDULE="guided,2" time -p ./para-for
# ! echo; echo guided, chunk=10:
# ! OMP_SCHEDULE="guided,10" time -p ./para-for

static:
real 23.26
user 31.36
sys 0.01

dynamic:
real 23.50
user 31.76
sys 0.01

guided:
real 23.66
user 31.92
sys 0.01
